# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate as tc
from turicreate import SFrame

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = tc.SFrame('home_data.sframe/')

In [3]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [5]:
sales

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2.0,0


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [6]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [7]:
model_all = tc.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.235247     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.306237     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [8]:
rows = len(all_features)
model_all.coefficients.print_rows(num_rows=rows)


+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

In [9]:
non_zero_weight = model_all.coefficients[model_all.coefficients["value"] != 0]

In [10]:
non_zero_weight.print_rows(num_rows=rows)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|      grade       |  None | 842.0680348976231  |  None  |
|    sqft_above    |  None | 20.024722417091112 |  None  |
+------------------+-------+--------------------+--------+
[6 rows x 4 columns]



In [11]:
non_zero_weight

name,index,value,stderr
(intercept),None,274873.05595049576,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
grade,None,842.0680348976231,None
sqft_above,None,20.024722417091112,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [13]:
import numpy as np

a = np.logspace(1, 7, num=13)# crea los valores 10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7
nn = len(a)
error_vector = np.empty(13)

for i in range(nn):
    model= tc.linear_regression.create(training, target = 'price', features = all_features,
                                       l1_penalty=a[i],
                                       l2_penalty=0,
                                       validation_set = None,verbose=False)
    predict = model.predict(validation)
    errors = validation['price'] - predict
    square_errors = errors ** 2
    RSS = square_errors.sum()
    error_vector[i] = RSS
        
print (error_vector)

[6.25766285e+14 6.25766285e+14 6.25766286e+14 6.25766288e+14
 6.25766295e+14 6.25766317e+14 6.25766387e+14 6.25766607e+14
 6.25767303e+14 6.25769508e+14 6.25776518e+14 6.25799063e+14
 6.25883719e+14]


In [14]:
error_vector = np.array(error_vector)
a = np.array(a)
indice = error_vector.tolist().index(min(error_vector))
#print(indice, min(error_vector), a[indice])
print ('L1 penalty with low RSS in validation set :' + str(a[indice]))
print ('RSS low :' + str(min(error_vector)))

L1 penalty with low RSS in validation set :10.0
RSS low :625766285142460.5


***QUIZ QUESTION.*** What was the best value for the `l1_penalty`?
***R/ 10***

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?
**R/ 18**

In [15]:
model_L1= tc.linear_regression.create(training, target = 'price', features = all_features,
                                       l1_penalty=10.0,
                                       l2_penalty=0,
                                       validation_set = None,verbose=False)

In [16]:
non_zero_weight = model_L1.coefficients[model_L1.coefficients["value"] != 0]

In [17]:
non_zero_weight.print_rows(num_rows=rows)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values = np.logspace(8, 10, num=20)#10^8,.......10^10

In [20]:
l1_penalty_values

array([1.00000000e+08, 1.27427499e+08, 1.62377674e+08, 2.06913808e+08,
       2.63665090e+08, 3.35981829e+08, 4.28133240e+08, 5.45559478e+08,
       6.95192796e+08, 8.85866790e+08, 1.12883789e+09, 1.43844989e+09,
       1.83298071e+09, 2.33572147e+09, 2.97635144e+09, 3.79269019e+09,
       4.83293024e+09, 6.15848211e+09, 7.84759970e+09, 1.00000000e+10])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [21]:
a = np.logspace(8, 10, num=20)# crea los valores 10^8, ..., 10^10
nn = len(a)
coef_dict = np.empty(20)

for i in range(nn):
    model= tc.linear_regression.create(training, target = 'price', features = all_features,
                                       l1_penalty=a[i],
                                       l2_penalty=0,
                                       validation_set = None,verbose=False)
    coef_dict[i] = model.coefficients["value"].nnz()
        
print (coef_dict)

[18. 18. 18. 18. 17. 17. 17. 17. 17. 16. 15. 15. 13. 12. 10.  6.  5.  3.
  1.  1.]


In [22]:
print (str(a) + ': '+ str(coef_dict))

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]: [18. 18. 18. 18. 17. 17. 17. 17. 17. 16. 15. 15. 13. 12. 10.  6.  5.  3.
  1.  1.]


In [23]:
frame = tc.SFrame({ 'l1_penalty' : a , 'coef_nonzeros' : coef_dict })

In [24]:
frame.print_rows(num_rows=20)

+---------------+--------------------+
| coef_nonzeros |     l1_penalty     |
+---------------+--------------------+
|      18.0     |    100000000.0     |
|      18.0     | 127427498.57031322 |
|      18.0     | 162377673.91887242 |
|      18.0     |  206913808.111479  |
|      17.0     | 263665089.87303555 |
|      17.0     | 335981828.6283788  |
|      17.0     | 428133239.8719396  |
|      17.0     | 545559478.1168514  |
|      17.0     | 695192796.1775591  |
|      16.0     | 885866790.4100832  |
|      15.0     | 1128837891.6846883 |
|      15.0     | 1438449888.2876658 |
|      13.0     | 1832980710.8324375 |
|      12.0     | 2335721469.0901213 |
|      10.0     | 2976351441.6313133 |
|      6.0      | 3792690190.7322536 |
|      5.0      | 4832930238.571753  |
|      3.0      | 6158482110.6602545 |
|      1.0      | 7847599703.514623  |
|      1.0      |   10000000000.0    |
+---------------+--------------------+
[20 rows x 2 columns]



Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

**TRADUCCION**

Fuera de este amplio rango, queremos encontrar los dos extremos de nuestro rango angosto deseado de `l1_penalty`. En un extremo, tendremos valores `l1_penalty` que tienen muy pocos valores distintos de cero y, en el otro extremo, tendremos un valor `l1_penalty` que tiene demasiados valores distintos de cero.

Más formalmente, encuentra:

* La `l1_penalty` más grande que tiene más valores distintos de cero que `max_nonzeros` (si elegimos una penalización más pequeña que este valor, definitivamente tendremos demasiados pesos distintos de cero)
    * Guarde este valor en la variable `l1_penalty_min` (lo usaremos más adelante)
* La l1_penalty más pequeña que tiene menos valores distintos de cero que `max_nonzeros` (si elegimos una penalización mayor que este valor, definitivamente tendremos muy pocos pesos distintos de cero)
    * Guarde este valor en la variable `l1_penalty_max` (lo usaremos más adelante)
*Sugerencia: hay muchas maneras de hacer esto, por ejemplo:*

* Programáticamente dentro del bucle de arriba
* Crear una lista con el número de valores distintos de cero para cada valor de `l1_penalty` e inspeccionarla para encontrar los límites apropiados.

In [25]:
l1_penalty_min = frame['l1_penalty'][14]
l1_penalty_max = frame['l1_penalty'][15]

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [26]:
print (l1_penalty_min)
print (l1_penalty_max)

2976351441.6313133
3792690190.7322536


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [27]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [28]:
l1_penalty_values

array([2.97635144e+09, 3.01931664e+09, 3.06228184e+09, 3.10524703e+09,
       3.14821223e+09, 3.19117743e+09, 3.23414263e+09, 3.27710782e+09,
       3.32007302e+09, 3.36303822e+09, 3.40600341e+09, 3.44896861e+09,
       3.49193381e+09, 3.53489901e+09, 3.57786420e+09, 3.62082940e+09,
       3.66379460e+09, 3.70675980e+09, 3.74972499e+09, 3.79269019e+09])

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [29]:
a = 0
a = np.linspace(l1_penalty_min,l1_penalty_max,20)# crea los valores 
nn = len(a)
error_vector = np.empty(20)
coef_dict = np.empty(20)

for i in range(nn):
    model= tc.linear_regression.create(training, target = 'price', features = all_features,
                                       l1_penalty=a[i],
                                       l2_penalty=0,
                                       validation_set = None,verbose=False)
    predictions = model.predict(validation)
    errors = validation['price'] - predictions
    square_errors = errors ** 2
    RSS = square_errors.sum()
    error_vector[i] = RSS
    coef_dict[i] = model.coefficients["value"].nnz() 
        
print (error_vector)

[9.66925692e+14 9.74019450e+14 9.81188368e+14 9.89328342e+14
 9.98783211e+14 1.00847717e+15 1.01829878e+15 1.02824799e+15
 1.03461691e+15 1.03855474e+15 1.04323724e+15 1.04693749e+15
 1.05114763e+15 1.05599274e+15 1.06079953e+15 1.06570769e+15
 1.06946434e+15 1.07350455e+15 1.07763278e+15 1.08186759e+15]


In [30]:
frame = tc.SFrame({ 'l1_penalty' : a , 'coef_nonzeros' : coef_dict, 'RSS' : error_vector })

In [31]:
frame.print_rows(num_rows=20)

+--------------------+---------------+--------------------+
|        RSS         | coef_nonzeros |     l1_penalty     |
+--------------------+---------------+--------------------+
| 966925692362085.2  |      10.0     | 2976351441.6313133 |
| 974019450084554.6  |      10.0     | 3019316638.9524155 |
| 981188367942453.8  |      10.0     | 3062281836.2735176 |
| 989328342459473.5  |      10.0     | 3105247033.5946198 |
| 998783211265890.1  |      10.0     | 3148212230.915722  |
| 1008477167020094.4 |      10.0     | 3191177428.236824  |
| 1018298780553819.4 |      10.0     | 3234142625.557926  |
| 1028247992205977.0 |      10.0     | 3277107822.8790283 |
| 1034616909232827.4 |      8.0      | 3320073020.2001305 |
| 1038554735941040.2 |      8.0      | 3363038217.5212326 |
| 1043237237871701.2 |      8.0      | 3406003414.8423347 |
| 1046937488751711.5 |      7.0      | 3448968612.163437  |
| 1051147625612862.5 |      7.0      | 3491933809.484539  |
| 1055992735342999.4 |      7.0      | 3

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [32]:
frame_min = frame[frame['coef_nonzeros']==7]
frame_min

RSS,coef_nonzeros,l1_penalty
1046937488751711.5,7.0,3448968612.163437
1051147625612862.5,7.0,3491933809.484539
1055992735342999.4,7.0,3534899006.805641
1060799531763286.9,7.0,3577864204.126743


In [33]:
frame_min['RSS'].min()

1046937488751711.5

In [34]:
valor_menor = frame_min[frame_min['RSS'] == frame_min['RSS'].min() ]


In [35]:
valor_menor

RSS,coef_nonzeros,l1_penalty
1046937488751711.5,7.0,3448968612.163437


In [36]:
model= tc.linear_regression.create(training, target = 'price', features = all_features,
                                       l1_penalty=valor_menor['l1_penalty'][0],
                                       l2_penalty=0,
                                       validation_set = None,verbose=False)

In [37]:
non_zero_weight = model.coefficients[model.coefficients["value"] != 0]

In [38]:
non_zero_weight.print_rows(num_rows=rows)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.1925443279  |  None  |
|     bedrooms     |  None | 661.7227177822278  |  None  |
|    bathrooms     |  None |  15873.957259268   |  None  |
|   sqft_living    |  None | 32.41022145125974  |  None  |
| sqft_living_sqrt |  None | 690.1147733133166  |  None  |
|      grade       |  None | 2899.420269749878  |  None  |
|    sqft_above    |  None | 30.011575302201138 |  None  |
+------------------+-------+--------------------+--------+
[7 rows x 4 columns]

